In [59]:
# This file contains all of the efficiency functions necessary for the power model
# Make sure ALL FUNCTIONS from this are imported to the main you need it in

# libraries
import numpy
import math

In [84]:
def position_eff_func(theta, pos_err, point_err, F_disp, h, r):
    # This function determines the efficiency associated with the error in pointing and position
    # error values given to this function are kept constant and positive, allowing those to be varied via calling this func
    # takes heritage valaues for attitude and position knowledge tolerances
    
    
    # define errors in each dimension
    xpos_err = pos_err[0];
    ypos_err = pos_err[1];
    hpos_err = pos_err[2];
    theta_err = point_err[0];
    phi_err = point_err[1];

    
    # Define/calculate other necessary values, taking errors in
    dx = 0.01; # m, linear step size
    dA = dx**2; # m2, area step size
    x = h*numpy.tan(theta);
    d = math.sqrt((x+xpos_err)**2 + (h+hpos_err)**2)
    FOV = 2*(numpy.arctan((x+xpos_err+r)/(h+hpos_err))-numpy.arctan((x+xpos_err)/(h+hpos_err)))
    r_prime = d*numpy.arctan(FOV/2)
    
    # find total errors
    
    xtheta_err = h*numpy.tan(theta+theta_err)-x;
    x_err = xpos_err + xtheta_err;
    
    yphi_err = d*numpy.tan(phi_err);
    y_err = ypos_err + yphi_err;
    
    # define shell radii, & their respective flux %
    shell_num = len(F_disp[0]);
    
    r_shell = F_disp[0];
    Fperc_shell = F_disp[1];
    
    # Preallocation
    A_hit = numpy.zeros(shell_num);
    A_avail = numpy.zeros(shell_num);
    hit_eff = numpy.zeros(shell_num);
    shell_eff = numpy.zeros(shell_num);
    
    # Loop through shells
    for shell_index in range(0,shell_num-1):
        if shell_index == 0:
            r_outer = r_shell[shell_index];
            r_inner = 0;
        else:
            r_outer = r_shell[shell_index];
            r_inner = r_shell[shell_index-1];
                    
        r_outer = int(r_outer);
        r_inner = int(r_inner);
        
        # loop through the total x of the shell
        for x in numpy.arange(-r_outer,r_outer,dx):
            
            # loop through the max and min y at x, not including within the shell
            y_max = math.sqrt(r_outer**2 - x**2);
            
            if abs(x) > abs(r_inner):
                # before scanning through the inner shell
            
                for y in numpy.arange(-y_max,y_max,dx):
                
                    # check if point is on the receiver
                    hit_value = ((x-x_err)**2)/(r_prime**2) + ((y-y_err)**2)/(r**2) 
                
                    if hit_value < 1:
                        A_hit[shell_index] = A_hit[shell_index] + dA

            else:
                # scans top and bottom parts of the shell
                
                y_inner = math.sqrt(r_inner**2 - x**2)
                
                for y in numpy.arange(-y_max,-y_inner,dx):
                
                    # check if point is on the receiver
                    hit_value = ((x-x_err)**2)/(r_prime**2) + ((y-y_err)**2)/(r**2) 
                
                    if hit_value < 1:
                        A_hit[shell_index] = A_hit[shell_index] + dA
                        
                for y in numpy.arange(y_inner,y_max,dx):
                
                    # check if point is on the receiver
                    hit_value = ((x-x_err)**2)/(r_prime**2) + ((y-y_err)**2)/(r**2) 
                
                    if hit_value < 1:
                        A_hit[shell_index] = A_hit[shell_index] + dA
                
        # now, per shell, find out efficiency
        
        A_avail[shell_index] = math.pi*r_outer**2 - math.pi*r_inner**2
        
        hit_eff[shell_index] = A_hit[shell_index]/A_avail[shell_index] # if this is close to 1, then the shell is 100% on the receiver, even with error
    
        # now, figure out overall percentage of flux that 
        
        shell_eff[shell_index] = hit_eff[shell_index]*Fperc_shell[shell_index]
        
    # sum different flux percentages to get total percentage
    total_eff = sum(shell_eff)
    
    return total_eff

position_eff_func(0, [0,0,0], [0,0], [[1,2,3,4],[70,20,7,3]], 30000, 4)
                                      

97.27408367776059

In [62]:
def receiver_eff_func(theta, F):
    # This function determines the efficiency of a receiver as a function of incident angle and flux
    # will probably be a combination of experimental values and functions 
    return 0.2 # example value